In [2]:
!pip install fuzzywuzzy


In [3]:
!pip install python-Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.0 MB/s eta 0:00:00


In [4]:
# Import necessary libraries
import numpy as np  # For numerical operations
import pandas as pd  # For data manipulation
from fuzzywuzzy import fuzz  # For fuzzy string matching
from fuzzywuzzy import process  # For advanced fuzzy operations


In [5]:
# Load the dataset
df = pd.read_csv('tested.csv')  # Replace 'test.csv' with your dataset filename

In [6]:
# Display the first few rows of the dataset
print("Dataset Preview:")
print(df.head())

# Display dataset information
print("\nDataset Information:")
print(df.info())

# Display summary statistics
print("\nDataset Summary Statistics:")
print(df.describe(include='all'))

Dataset Preview:
   PassengerId  Survived  Pclass  \
0          892         0       3   
1          893         1       3   
2          894         0       2   
3          895         0       3   
4          896         1       3   

                                           Name     Sex   Age  SibSp  Parch  \
0                              Kelly, Mr. James    male  34.5      0      0   
1              Wilkes, Mrs. James (Ellen Needs)  female  47.0      1      0   
2                     Myles, Mr. Thomas Francis    male  62.0      0      0   
3                              Wirz, Mr. Albert    male  27.0      0      0   
4  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female  22.0      1      1   

    Ticket     Fare Cabin Embarked  
0   330911   7.8292   NaN        Q  
1   363272   7.0000   NaN        S  
2   240276   9.6875   NaN        Q  
3   315154   8.6625   NaN        S  
4  3101298  12.2875   NaN        S  

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeInde

In [7]:
# Check for missing values
print("\nMissing Values in the Dataset:")
print(df.isnull().sum())

# Display columns with missing values
columns_with_na = df.columns[df.isnull().any()]
print("\nColumns with Missing Values:", columns_with_na)



Missing Values in the Dataset:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Columns with Missing Values: Index(['Age', 'Fare', 'Cabin'], dtype='object')


In [8]:
# Handle missing values
df['Age'].fillna(df['Age'].mean(), inplace=True)  # Fill missing 'Age' with mean
df['Fare'].fillna(df['Fare'].median(), inplace=True)  # Fill missing 'Fare' with median
df.drop('Cabin', axis=1, inplace=True)  # Drop 'Cabin' column due to excessive missing data

# Verify no missing values remain
print("\nMissing Values After Processing:")
print(df.isnull().sum())



Missing Values After Processing:
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


<ipython-input-8-91e6e71abc8c>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].mean(), inplace=True)  # Fill missing 'Age' with mean
<ipython-input-8-91e6e71abc8c>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(v

In [9]:
# Display unique values in specific columns
print("\nUnique Values in Embarked Column:")
print(df['Embarked'].unique())

# Count distribution of a specific column
print("\nValue Counts for 'Survived':")
print(df['Survived'].value_counts())



Unique Values in Embarked Column:
['Q' 'S' 'C']

Value Counts for 'Survived':
Survived
0    266
1    152
Name: count, dtype: int64


In [10]:
# Function to find potential duplicates using fuzzy matching
def find_duplicates(dataframe, column, threshold=80):
    duplicates = []
    for i, name in enumerate(dataframe[column]):
        for j, other_name in enumerate(dataframe[column]):
            if i != j:  # Avoid self-comparison
                similarity = fuzz.token_sort_ratio(name, other_name)
                if similarity >= threshold:
                    duplicates.append((name, other_name, similarity))
    return duplicates

# Apply the function to the 'Name' column
potential_duplicates = find_duplicates(df, 'Name', threshold=80)

# Display potential duplicates
print("\nPotential Duplicates Found:")
for duplicate in potential_duplicates[:10]:  # Show only the first 10 for brevity
    print(duplicate)



Potential Duplicates Found:
('Brady, Mr. John Bertram', 'Crafton, Mr. John Bertram', 82)
('Davison, Mr. Thomas Henry', 'Conlon, Mr. Thomas Henry', 80)
('Kiernan, Mr. John', 'Kennedy, Mr. John', 80)
('Kiernan, Mr. John', 'Lingane, Mr. John', 80)
('Kennedy, Mr. John', 'Kiernan, Mr. John', 80)
('Crafton, Mr. John Bertram', 'Brady, Mr. John Bertram', 82)
('Lingane, Mr. John', 'Kiernan, Mr. John', 80)
('Dennis, Mr. William', 'Dibden, Mr. William', 82)
('Dibden, Mr. William', 'Dennis, Mr. William', 82)
('Dibden, Mr. William', 'Gilbert, Mr. William', 80)


In [11]:
# Deduplicate dataset by retaining unique names
unique_names = set()
deduplicated_rows = []

for _, row in df.iterrows():
    name = row['Name']
    if name not in unique_names:
        deduplicated_rows.append(row)
        unique_names.add(name)

# Create a new DataFrame with deduplicated rows
df_cleaned = pd.DataFrame(deduplicated_rows)

# Check the shape of the cleaned dataset
print("\nShape of Cleaned Dataset:")
print(df_cleaned.shape)



Shape of Cleaned Dataset:
(418, 11)


In [13]:
# Save the cleaned dataset
df_cleaned.to_csv('cleaned_data.csv', index=False)

print("\nCleaned dataset has been saved as 'cleaned_data.csv'.")



Cleaned dataset has been saved as 'cleaned_data.csv'.
